In [ ]:
def check_dir(directory):
    if not os.path.exists(directory):
        print('ERROR!!! DIRECTORY DOESNT EXISTS -', directory)
    else:
        print('DIRECTORY EXISTS -', directory)
    print()
    
def check_and_create_dir(directory):
    if not os.path.exists(directory):
        print('DIRECTORY DOESNT EXISTS -', directory)
        os.mkdir(directory)
        print()
        print('DIRECTORY CREATED -', directory)
    else:
        print('DIRECTORY EXISTS -', directory)
    print()

In [ ]:
import os
base_folder = "/Users/aishwaryamalgonde/Aishwarya/nanonets"
os.chdir(base_folder)
os.getcwd()

In [ ]:
parent_folder = '46_n' # component type

In [ ]:
data = "/Users/aishwaryamalgonde/Aishwarya/nanonets/data/split_typec/" + parent_folder

run_model = "vgg16"

import os
# main_dir = os.path.join(parent_folder, run_model)
check_and_create_dir(parent_folder)

In [ ]:
model_dir = os.path.join(parent_folder, run_model)#, "")
check_and_create_dir(model_dir)

os.chdir(model_dir)
os.getcwd()

In [ ]:
# DATA_FOLDER = '/Users/aishwaryamalgonde/Aishwarya/nanonets/data/split_typec/6/6/training_data'

DATA_DIR_TRAIN = os.path.join( data, "train")
DATA_DIR_VAL = os.path.join(data, "test")

check_dir(DATA_DIR_TRAIN)
check_dir(DATA_DIR_VAL)

# OUTPUT_DIR = os.path.join(os.getcwd(), "output")
# check_and_create_dir(OUTPUT_DIR)

# /Users/aishwaryamalgonde/Aishwarya/nanonets/data/split_typec/20_f/test

# 2. Initialization

In [ ]:
import tensorflow as tf
tf.__version__

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import datetime

from keras.layers import Dense, GlobalAveragePooling2D, Dropout, Flatten
from keras.applications import VGG16
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.initializers import glorot_uniform

from tensorflow.keras import backend as K
K.image_data_format()
K.set_image_data_format('channels_last')
K.image_data_format()

# 3. Model Building

In [ ]:
dropout_rate = 0.3

if run_model == "xception" or run_model == "inceptionv3":
    img_h = 229
    img_w = 229
else:
    img_h = 224
    img_w = 224
    
img_c = 3

batch_size = 1

input_shape = (img_h, img_w, img_c)
input_shape_hw = (img_h, img_w)

checkpoint_file = run_model + "_best_ld"

print("batch_size : ", batch_size)
print("img_h : ", img_h)
print("img_w : ", img_w)

In [ ]:
if run_model == "vgg16": #     (224, 224, 3) (224,224)
    print("Model : VGG16")
    base_model = VGG16(include_top=False, weights='imagenet', input_shape=input_shape)
    
else:
    print("ERROR!!! Please select appropriate model name.")

In [ ]:
x=base_model.output

# x=GlobalAveragePooling2D()(x)

x = Flatten()(x)

# x=Dropout(dropout_rate)(x)

x = Dense(1024, 
        activation='relu',
#         kernel_regularizer=regularizers.l2(0.01), 
        kernel_initializer=glorot_uniform(seed=None))(x) #we add dense layers so that the model can learn more complex functions and classify for better results.

x = Dropout(dropout_rate)(x)

# x=Dense(1024,activation='relu')(x) #dense layer 2

# x=Dense(512,activation='relu')(x) #dense layer 3

# x=Dropout(dropout_rate)(x)

preds=Dense(1,
            activation='sigmoid',
#             kernel_regularizer=regularizers.l2(0.01),
            kernel_initializer=glorot_uniform(seed=None))(x) #final layer with softmax activation

In [ ]:
model=Model(inputs=base_model.input,outputs=preds)
#specify the inputs
#specify the outputs
#now a model has been created based on our architecture

In [ ]:
for i,layer in enumerate(model.layers):
    print(i,layer.name)

In [ ]:
# 87 for MobileNet
# 780 for InceptionResNetv2
# 19 for vgg16
# 132 for Xception
# 178 resnet50
n = 19
for layer in model.layers[:n]:
    layer.trainable=False

print("TRAINABLE LAYERS")
print()
for layer in model.layers[n:]:
    print(layer.name)
    layer.trainable=True

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

# Augmentation
# zoom_range = 0.4
rotation_range = 10
vflip = True
# hflip = True
# width_shift_range=0.2,
# height_shift_range=0.2,
shear_range=0.05,
# width_shift_range = 0.3
# height_shift_range = 0.3
# preprocessing
rescale = 1. / 255.
preprocessing_function = None

# seed = 20170804
# batch_size = 5
    
# creating dictionary
data_gen_args = dict(
    featurewise_center=False,
    featurewise_std_normalization=False,
    rescale=rescale,
    preprocessing_function=preprocessing_function,
    shear_range=shear_range,
#     zoom_range=zoom_range,
    rotation_range=rotation_range,
#     width_shift_range=width_shift_range,
#     height_shift_range=height_shift_range,
    vertical_flip=vflip,
#     horizontal_flip=hflip)

# *args and **kwargs is a common idiom to allow arbitrary number of arguments to functions 
image_datagen = ImageDataGenerator(**data_gen_args)

val_datagen = ImageDataGenerator(rescale = rescale)

In [ ]:
train_generator=image_datagen.flow_from_directory(DATA_DIR_TRAIN, # this is where you specify the path to the main data folder
                                                 target_size=input_shape_hw,
                                                 color_mode='rgb',
                                                 batch_size=batch_size,
                                                 class_mode='binary',
                                                 shuffle=True)

val_generator=val_datagen.flow_from_directory(DATA_DIR_VAL, # this is where you specify the path to the validation data folder
                                              target_size=input_shape_hw,
                                              color_mode='rgb',
                                              batch_size=batch_size,
                                              class_mode='binary',
                                             shuffle=True)

In [ ]:
# Adam optimizer
opt = Adam(lr=0.00001)

# loss function will be categorical cross entropy
# evaluation metric will be accuracy
model.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
# CALCULATING STEP SIZE FOR TRAIN AND VAL DATASET
step_size_train = np.ceil(train_generator.n/train_generator.batch_size)
step_size_val = np.ceil(val_generator.n/val_generator.batch_size)
step_size_train, step_size_val

In [ ]:
TRAIN_EPOCHS = 20

In [ ]:
# initialize the decay for the optimizer
# decay = 0.0
# #opt_sgd = keras.optimizers.SGD(lr=TRAIN_MAX_LR, decay=decay, momentum=0.9)
# model.compile(optimizer=opt,
#               loss='binary_crossentropy',
#               metrics=['accuracy'])

# log_dir="logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
# tensorboard_callback = TensorBoard(log_dir="logs")

# SAVING THE CHECKPOINT WITH LOWEST VAL_LOSS
checkpoint = ModelCheckpoint(checkpoint_file, 
                             monitor='val_loss', 
                             verbose=1, 
                             save_best_only=True, 
                             mode='min', 
                             period=1) 
                            #, save_weights_only=True)

# train the network
print("[INFO] training network...")

H = model.fit_generator(generator = train_generator,
                        steps_per_epoch = step_size_train,
                        validation_data = val_generator,                              
                        validation_steps = step_size_val,
                        verbose = 1,epochs = TRAIN_EPOCHS,
                        callbacks=[checkpoint])

In [ ]:
# train_plot = os.path.join(os.getcwd(), "output", "training_"+schedule_type+".png")

# # plot the training loss and accuracy
# N = np.arange(0, TRAIN_EPOCHS)
# plt.style.use("ggplot")
# plt.figure()
# plt.plot(N, H.history["loss"], label="train_loss")
# plt.plot(N, H.history["val_loss"], label="val_loss")
# plt.plot(N, H.history["accuracy"], label="train_acc")
# plt.plot(N, H.history["val_accuracy"], label="val_acc")
# plt.title("Training Loss and Accuracy on CIFAR-10")
# plt.xlabel("Epoch #")
# plt.ylabel("Loss/Accuracy")
# plt.legend()
# plt.savefig(train_plot)

In [ ]:
# acc_plot = os.path.join(os.getcwd(), "output", "acc_"+schedule_type+".png")
# loss_plot = os.path.join(os.getcwd(), "output", "loss_"+schedule_type+".png")
# lr_plot = os.path.join(os.getcwd(), "output", "lr_"+schedule_type+".png")

# # plot the training loss and accuracy
# N = np.arange(0, TRAIN_EPOCHS)
# plt.style.use("ggplot")
# plt.figure()
# plt.plot(N, H.history["accuracy"], label="train_acc")
# plt.plot(N, H.history["val_accuracy"], label="val_acc")
# plt.title("Training Accuracy")
# plt.xlabel("Epoch #")
# plt.ylabel("Accuracy")
# plt.legend()
# plt.savefig(acc_plot)

# plt.style.use("ggplot")
# plt.figure()
# plt.plot(N, H.history["loss"], label="train_loss")
# plt.plot(N, H.history["val_loss"], label="val_loss")
# plt.title("Training Loss")
# plt.xlabel("Epoch #")
# plt.ylabel("Loss")
# plt.legend()
# plt.savefig(loss_plot)

# if schedule is not None:
# 	schedule.plot(N)
# 	plt.savefig(lr_plot)

# Loading Model

## Method 2

In [ ]:
from keras import backend as K
# This line must be executed before loading Keras model.
K.set_learning_phase(0)

from keras.models import load_model
model2 = load_model(checkpoint_file)
print("[INFO] Loaded model from disk")

print(model2.outputs)
print(model2.inputs)

In [ ]:
from keras.preprocessing import image

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

# Augmentation
# preprocessing
rescale = 1. / 255.
preprocessing_function = None

# seed = 20170804
# batch_size = 5
    
# creating dictionary
data_gen_args = dict(rescale=rescale,
    preprocessing_function=preprocessing_function)

# *args and **kwargs is a common idiom to allow arbitrary number of arguments to functions 
# image_datagen = ImageDataGenerator(**data_gen_args)

test_datagen = ImageDataGenerator(rescale = rescale)

In [ ]:
# image_path = '/Users/aishwaryamalgonde/Aishwarya/nanonets/data/split_typec/6/6/0/10.jpg_6_6_0.jpg'
image_path = ('/Users/aishwaryamalgonde/Aishwarya/nanonets/data/split_typec/'+
              parent_folder+
              '/correct/10.jpg_20_f_0.jpg')

In [ ]:
# predicting images
img_width = 224
img_height = 224
batch_size = 1
img = image.load_img(image_path, target_size=(img_width, img_height))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = x/255.
# x

In [ ]:
# images = np.vstack([x])
classes = model2.predict(x)
classes[0][0]

In [ ]:
input_shape_hw = (img_height, img_width)

DATA_DIR_TEST = "/Users/aishwaryamalgonde/Aishwarya/nanonets/data/split_typec/6_6/training_data/test"

## READING THE TEST DIRECTORY
test_generator = test_datagen.flow_from_directory(
        DATA_DIR_TEST,
        target_size=input_shape_hw,
        batch_size=batch_size,
        class_mode=None,  # only data, no labels
        shuffle=False) 
test_generator.classes

In [ ]:
correct = 0
rotated = 1